In [1]:
!pip install gym 

In [2]:
!pip install atari-py

In [3]:
!pip install gym[atari]

# Implementation details

There are 5 modules of Deep Q learning-
1. CNN for interacting with the environment
2. Experience replay so that gradient descent converges rather than diverges. It basically provides a sense of direction to where to move for reducing the loss
3. A module to integrate both of the above modules and building a linking peice to complete the algorithm
4. testing with the OpenAI gym environment
5. Preprocessing of image

# import statements

In [0]:
import gym
import numpy as np
from skimage.transform import resize
import tensorflow as tf

# Initializing Gym environment

In [0]:
env = gym.make("Breakout-v4")
env = env.unwrapped
env.seed(1)
state = env.reset()

# Pre processing of image

In [0]:
def frame_preprocessing(image_frame):
  image_frame=resize(image_frame, (84,84,3))
  grayscale_frame=np.dot(image_frame[...,:3], [0.2989, 0.5870, 0.1140])
  return grayscale_frame

# Experience replay

## Memory class

In [0]:
class memory_frame(object):
  def __init__(self,state,action,reward):
    self.state=state
    self.action=action
    self.reward=reward

In [0]:
experience_memory=list()

# Hyper parameters

In [0]:
action_number=env.action_space.n
learning_rate=0.01

# Q Network

The Q network architecture-
1. 16 filters with 8 X 8 kernel and 4 strides with relu activation
2. 32 filters with 4 X 4 kernel and 2 strides with relu activation
3. 256 fully connected layer
4. linear layer with number of actions

In [0]:
tf.compat.v1.disable_eager_execution()
input_state=tf.compat.v1.placeholder(tf.float32, shape=[None, 84,84,1],name="input_state")
y_j=tf.compat.v1.placeholder(tf.float32, shape=[None,],name="y_j")
cnn_layer_1=tf.keras.layers.Conv2D(filters=16,kernel_size=(8,8),strides=(4,4),activation="relu")(input_state)
cnn_layer_2=tf.keras.layers.Conv2D(filters=32,kernel_size=(4,4),strides=(2,2),activation="relu")(cnn_layer_1)
flatten_layer=tf.keras.layers.Flatten()(cnn_layer_2)
fully_connected_layer_1=tf.keras.layers.Dense(256,activation='relu',name="fc2")(flatten_layer)
output_layer=tf.keras.layers.Dense(action_number,activation='linear',name="fc2")(fully_connected_layer_1)
loss=tf.math.reduce_mean((y_j-output_layer)**2)
training=tf.compat.v1.train.AdamOptimizer(learning_rate).minimize(loss)